# 分类模型预测结果分析

## 验证数据集加载

In [1]:
import sys
import json
import os
sys.path.append("..")
import utils
import utils.dataloader
import utils.transforms

In [2]:
val_path =  "/root/autodl-tmp/seat_dataset/chengdu_valid/"
val_ann =  "/root/autodl-tmp/seat_task/jobs/job_8/output/_annclassfication.valid.coco.json"

In [3]:
with open(val_ann, 'r') as f:
    val_annotations = json.load(f)

val_annotations['annotations'] = [ ann for ann in val_annotations['annotations'] if ann['source'] == 'predict' ]

# 保存为指定的名称
with open('_val_annotations_predict_only.json', 'w') as f:
    json.dump(val_annotations, f, indent=2)

print("已保存过滤后的标注文件为: _val_annotations_predict_only.json")

已保存过滤后的标注文件为: _val_annotations_predict_only.json


In [4]:
val_ann = "_val_annotations_predict_only.json"

In [5]:
dataload, dataset = utils.dataloader.create_coco_dataloader(val_path, val_ann, utils.transforms.get_default_transform(is_train=False), min_bbox_area=0, crop_scale_factor=1.5, is_train=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


/root/miniconda3/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


## 2. 加载分类模型

In [6]:
from trainer import get_model

In [7]:
import torch

# 检查模型文件是否存在
model_path = "/root/autodl-tmp/seat_task/jobs/job_8/output/lightning_logs/version_0/checkpoints/best_loss_model_epoch=33_val_loss=0.6444.ckpt"

cls_config = {
    'check_point_path': model_path,
    "backbone_name": 'dinov3_vitb16',
    'REPO_DIR': '/root/dinov3',
    'num_classes': 2
}
cls_config['class_names'] = ['缺陷', '背景']

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

model = get_model(cls_config)
model.eval()
model.to(device)

使用设备: cuda
backbone name: dinov3_vitb16
backbone weights: None
Successfully loaded model weights from: /root/autodl-tmp/seat_task/jobs/job_8/output/lightning_logs/version_0/checkpoints/best_loss_model_epoch=33_val_loss=0.6444.ckpt


DinoV3Classifier(
  (backbone): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (rope_embed): RopePositionEmbedding()
    (blocks): ModuleList(
      (0-11): 12 x SelfAttentionBlock(
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): SelfAttention(
          (qkv): LinearKMaskedBias(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, i

In [ ]:
all_pred = []
all_label = []
for img, label, info in dataload:
    # 将输入数据移动到与模型相同的设备上
    img = img.to(device)
    pred = model(img)
    pred = torch.sigmoid(pred)
    all_pred.extend(pred.detach().cpu().numpy())
    all_label.extend(label.cpu().numpy())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.ndimage import gaussian_filter1d

plt.rcParams['font.sans-serif'] = [
    'Noto Sans CJK SC',
    'Noto Serif CJK SC',
    'WenQuanYi Zen Hei',
    'WenQuanYi Micro Hei',
    'SimHei', 'SimSun', 'FangSong'
]
plt.rcParams['axes.unicode_minus'] = False 

# 将预测结果转换为numpy数组
all_pred = np.array(all_pred).flatten()
all_label = np.array(all_label)

# 分离两个类别的预测概率
class_0_pred = all_pred[all_label == 0]  # 缺陷类别的预测概率
class_1_pred = all_pred[all_label == 1]  # 背景类别的预测概率

# 创建子图
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 第一个子图：概率密度分布折线图
if len(class_0_pred) > 0 and len(class_1_pred) > 0:
    # 计算直方图数据用于绘制折线图
    bins = np.linspace(0, 1, 51)
    hist_0, bin_edges_0 = np.histogram(class_0_pred, bins=bins, density=True)
    hist_1, bin_edges_1 = np.histogram(class_1_pred, bins=bins, density=True)
    
    # 计算bin中心点
    bin_centers = (bin_edges_0[:-1] + bin_edges_0[1:]) / 2
    
    # 应用高斯平滑
    sigma = 1.0  # 平滑参数，可以调整
    hist_0_smooth = gaussian_filter1d(hist_0, sigma=sigma)
    hist_1_smooth = gaussian_filter1d(hist_1, sigma=sigma)
    
    # 绘制平滑后的折线图
    ax1.plot(bin_centers, hist_0_smooth, 'r-', linewidth=2.5, label='缺陷类别', alpha=0.8)
    ax1.plot(bin_centers, hist_1_smooth, 'b-', linewidth=2.5, label='背景类别', alpha=0.8)
    
    ax1.set_xlabel('预测概率')
    ax1.set_ylabel('概率密度')
    ax1.set_title('两类预测概率密度分布')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, 1)
    
    # 添加统计信息
    ax1.text(0.05, 0.9, f'缺陷类别样本数: {len(class_0_pred)}', 
             transform=ax1.transAxes, fontsize=10, bbox=dict(boxstyle="round,pad=0.3", facecolor="lightcoral", alpha=0.8))
    ax1.text(0.05, 0.8, f'背景类别样本数: {len(class_1_pred)}', 
             transform=ax1.transAxes, fontsize=10, bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8))

# 第二个子图：数量分布折线图
if len(class_0_pred) > 0 and len(class_1_pred) > 0:
    # 计算直方图数据用于绘制折线图
    bins = np.linspace(0, 1, 51)
    hist_0, bin_edges_0 = np.histogram(class_0_pred, bins=bins, density=False)
    hist_1, bin_edges_1 = np.histogram(class_1_pred, bins=bins, density=False)
    
    # 计算bin中心点
    bin_centers = (bin_edges_0[:-1] + bin_edges_0[1:]) / 2
    
    # 应用高斯平滑
    sigma = 1.0  # 平滑参数，可以调整
    hist_0_smooth = gaussian_filter1d(hist_0, sigma=sigma)
    hist_1_smooth = gaussian_filter1d(hist_1, sigma=sigma)
    
    # 绘制平滑后的折线图
    ax2.plot(bin_centers, hist_0_smooth, 'r-', linewidth=2.5, label='缺陷类别', alpha=0.8)
    ax2.plot(bin_centers, hist_1_smooth, 'b-', linewidth=2.5, label='背景类别', alpha=0.8)
    
    ax2.set_xlabel('预测概率')
    ax2.set_ylabel('样本数量')
    ax2.set_title('两类预测概率数量分布')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(0, 1)
    
    # 添加统计信息
    ax2.text(0.05, 0.9, f'缺陷类别总样本数: {len(class_0_pred)}', 
             transform=ax2.transAxes, fontsize=10, bbox=dict(boxstyle="round,pad=0.3", facecolor="lightcoral", alpha=0.8))
    ax2.text(0.05, 0.8, f'背景类别总样本数: {len(class_1_pred)}', 
             transform=ax2.transAxes, fontsize=10, bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8))

plt.tight_layout()
plt.show()

# 打印统计信息
print(f"缺陷类别样本数: {len(class_0_pred)}")
print(f"背景类别样本数: {len(class_1_pred)}")
if len(class_0_pred) > 0:
    print(f"缺陷类别预测概率 - 均值: {np.mean(class_0_pred):.4f}, 标准差: {np.std(class_0_pred):.4f}")
if len(class_1_pred) > 0:
    print(f"背景类别预测概率 - 均值: {np.mean(class_1_pred):.4f}, 标准差: {np.std(class_1_pred):.4f}")

# 打印关键阈值信息
if len(class_0_pred) > 0 and len(class_1_pred) > 0:
    print(f"\n关键阈值分析:")
    for threshold in [0.1, 0.3, 0.5, 0.7, 0.9]:
        count_0 = np.sum(class_0_pred >= threshold)
        count_1 = np.sum(class_1_pred >= threshold)
        total_remaining = count_0 + count_1
        defect_ratio = count_0 / total_remaining if total_remaining > 0 else 0
        print(f"阈值 {threshold}: 剩余缺陷={count_0}, 剩余背景={count_1}, 缺陷占剩余比例={defect_ratio:.3f}")


In [ ]:
# 计算PR曲线的函数
from sklearn.metrics import precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
import numpy as np

def plot_pr_curve(y_true, y_scores, positive_label=1, title_suffix=""):
    """
    绘制PR曲线
    
    Args:
        y_true: 真实标签
        y_scores: 预测概率分数
        positive_label: 正例标签 (0表示缺陷为正例，1表示背景为正例)
        title_suffix: 标题后缀
    """
    # 计算PR曲线
    precision, recall, thresholds = precision_recall_curve(y_true, y_scores, pos_label=positive_label)
    average_precision = average_precision_score(y_true, y_scores, pos_label=positive_label)
    
    # 绘制PR曲线
    plt.figure(figsize=(10, 6))
    plt.plot(recall, precision, 'b-', linewidth=2, label=f'PR曲线 (AP = {average_precision:.3f})')
    plt.xlabel('召回率 (Recall)')
    plt.ylabel('精确率 (Precision)')
    plt.title(f'精确率-召回率曲线{title_suffix}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    
    # 添加一些关键点的标注
    # 找到最佳F1分数对应的点
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
    best_f1_idx = np.argmax(f1_scores)
    best_f1 = f1_scores[best_f1_idx]
    best_precision = precision[best_f1_idx]
    best_recall = recall[best_f1_idx]
    
    plt.plot(best_recall, best_precision, 'ro', markersize=8, label=f'最佳F1点 (F1={best_f1:.3f})')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 打印关键指标
    print(f"平均精确率 (AP): {average_precision:.4f}")
    print(f"最佳F1分数: {best_f1:.4f}")
    print(f"最佳F1对应的精确率: {best_precision:.4f}")
    print(f"最佳F1对应的召回率: {best_recall:.4f}")
    
    # 计算不同阈值下的性能指标
    print("\n不同阈值下的性能指标:")
    print("阈值\t精确率\t召回率\tF1分数")
    for i in range(0, len(thresholds), max(1, len(thresholds)//10)):  # 显示10个点
        thresh = thresholds[i]
        prec = precision[i]
        rec = recall[i]
        f1 = 2 * (prec * rec) / (prec + rec + 1e-8)
        print(f"{thresh:.3f}\t{prec:.3f}\t{rec:.3f}\t{f1:.3f}")
    
    return precision, recall, thresholds, average_precision, best_f1

# 绘制缺陷作为正例的PR曲线
print("=" * 50)
print("缺陷作为正例的PR曲线分析")
print("=" * 50)
plot_pr_curve(all_label, all_pred, positive_label=0, title_suffix=" (缺陷为正例)")

# 绘制背景作为正例的PR曲线
print("\n" + "=" * 50)
print("背景作为正例的PR曲线分析")
print("=" * 50)
plot_pr_curve(all_label, all_pred, positive_label=1, title_suffix=" (背景为正例)")
